# Capas convolucionales y de agrupación

Cargamos el dataset como se ha hecho en prácticas anteriores

In [14]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner

keras.utils.set_random_seed(23); input_dim = (28, 28, 1); num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train_val = np.expand_dims(x_train_val, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape)
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]

(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


Esta vez el hipermodelo incorpora capas convolucionales y de pooling.

Two convolutional layers with:

- Filters: filters in the first layer, 2*filters in the second.
- Kernel size: (3, 3)
- Activation: ReLU

Max pooling layers reduce spatial dimensions by (2, 2).

In [15]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(28, 28, 1)))
    filters = hp.Int("filters", min_value=8, max_value=64, step=2, sampling="log")

    M.add(keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Conv2D(2*filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Flatten())
    M.add(keras.layers.Dense(units=800, activation='relu'))
    M.add(keras.layers.Dense(10, activation='softmax'))

    opt = keras.optimizers.Adam(learning_rate=0.00168)
    
    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M
  def fit(self, hp, M, x, y, xy_val, **kwargs):
    factor = 0.3787; patience = 5

    reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)

    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)

    kwargs['callbacks'].extend([reduce_cb, early_cb])
    return M.fit(x, y, batch_size=256, epochs=100, validation_data=xy_val, **kwargs)

El proceso de cómputo de este entrenamiento es muy pesado (por eso solo vamos a hacer una trial).

In [18]:
tuner = keras_tuner.BayesianOptimization(
MyHyperModel(), objective="val_accuracy", max_trials=1, executions_per_trial=1,
overwrite=True, directory="/tmp", project_name="MNIST")

In [19]:
tuner.search(x_train, y_train, (x_val, y_val))

Trial 1 Complete [00h 02m 03s]
val_accuracy: 0.9114000201225281

Best val_accuracy So Far: 0.9114000201225281
Total elapsed time: 00h 02m 03s


In [20]:
tuner.results_summary(num_trials=1)

Results summary
Results in /tmp/MNIST
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
filters: 8
Score: 0.9114000201225281


In [22]:
num_models = 1
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
  values = best_hyperparameters[m].values
  score = best_models[m].evaluate(x_test, y_test, verbose=0)
  print(f'Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}')

Model 0: Hyperparameters: {'filters': 8} Loss: 0.5079 Precisión: 90.70%
